In [84]:
from langchain.prompts import ChatPromptTemplate

prompt_vietnam = (
	"# Hướng dẫn xây dựng Đồ thị Tri thức từ văn bản quy chế tiếng Việt\n"
	"## 1. Mục tiêu\n"
	"Bạn là một mô hình trích xuất thông tin chất lượng cao, được thiết kế để xây dựng đồ thị tri thức từ văn bản quy định, đặc biệt là quy chế đào tạo đại học bằng tiếng Việt.\n"
	"Hãy cố gắng trích xuất tối đa thông tin có trong văn bản **mà không thêm bất kỳ thông tin nào không được nêu rõ**.\n"
	"- **Node** là các thực thể hoặc khái niệm xuất hiện trong văn bản.\n"
	"- **Edge** là quan hệ giữa các node, phản ánh mối liên kết có trong nội dung văn bản.\n"
	"- Kết quả phải dễ hiểu, rõ ràng, có thể dùng để xây dựng hệ thống truy vấn đồ thị tri thức.\n\n"

	"## 2. Gán nhãn thực thể (Labeling Nodes)\n"
	"- Mỗi node phải có `id` là tên thực thể trích từ văn bản, và `type` là loại thực thể.\n"
	"- Nếu thực thể thuộc các loại thường gặp như: `CHƯƠNG`, `ĐIỀU`, `KHOẢN`, `CHỦ_THỂ`, `HOẠT_ĐỘNG`, `YÊU_CẦU`, `CÓ`, `LÀ`, `XẾP_LOẠI`, `KHÁI_NIỆM`, ... thì hãy dùng đúng loại đó.\n"
	"- Nếu thực thể không thuộc các loại trên, HÃY TỰ ĐẶT `type` mới phù hợp bằng tiếng Việt.\n"
	"- Không được dùng số làm `id`. `id` phải là tên nguyên văn hoặc dễ hiểu từ văn bản, ví dụ: \"Chương trình đào tạo\", \"Sinh viên\".\n"
	"- Nếu trong văn bản có mô tả hoặc đặc điểm đi kèm thực thể, hãy trích xuất dưới dạng `properties`.\n"
	"- Mỗi node có thể có thêm các thuộc tính khác như `tiêu_đề`, `tên`, `định_nghĩa`, `mô_tả`, `vai_trò`, `phạm_vi`, v.v.\n"
	"- Nếu thuộc tính không nằm trong danh sách gợi ý, bạn được phép tự đặt tên phù hợp (vẫn theo chuẩn viết thường bằng tiếng Việt có dấu gạch dưới).\n\n"

	"## 3. Gán nhãn quan hệ (Relationships)\n"
	"- Các `edges` phải mô tả chính xác mối liên hệ giữa thực thể nguồn (`source`) và thực thể đích (`target`).\n"
	"- `type` của edge phải là một quan hệ rõ ràng, ngắn gọn, bằng **tiếng Việt viết HOA có dấu gạch dưới**, ví dụ: `QUY_ĐỊNH`, `BAO_GỒM`, `PHẢI_HOÀN_THÀNH`, `LIÊN_QUAN_ĐẾN`, `XẾP_LOẠI`, `THỜI_GIAN`, `LÀM`, `THUỘC`, `THAM_CHIẾU`.\n"
	"- Không dùng tên quan hệ quá cụ thể theo ngữ cảnh tạm thời, ví dụ: `BỊ_CẢNH_BÁO_2_LẦN` là không hợp lệ. Hãy dùng chung như `BỊ_CẢNH_BÁO`.\n"
	"- Nếu quan hệ không nằm trong danh sách gợi ý, bạn được phép tự đặt tên phù hợp (vẫn theo chuẩn viết HOA bằng tiếng Việt có dấu gạch dưới).\n"
	"- Nếu có thông tin bổ sung về quan hệ (ví dụ: điều_kiện, quy_định_chi_tiết, v.v.), hãy thêm vào `properties` của quan hệ.\n"
	"- Nếu thuộc tính không nằm trong danh sách gợi ý, bạn được phép tự đặt tên phù hợp (vẫn theo chuẩn viết thường bằng tiếng Việt có dấu gạch dưới).\n\n"

	"## 4. Đồng nhất thực thể (Coreference Resolution)\n"
	"- Nếu một thực thể xuất hiện nhiều lần với cách gọi khác nhau (VD: \"Trường\", \"Nhà Trường\", \"Trường Đại học Quy Nhơn\", \"trường\"), hãy dùng tên đầy đủ nhất làm ID duy nhất.\n"
	"- Không tạo trùng node cho cùng một thực thể.\n\n"

	"## 5. Tuân thủ định dạng và chính xác\n"
	"- Không giải thích, không thêm bình luận.\n"
	"- Chỉ trả về kết quả theo định dạng JSON gồm `nodes` và `edges`.\n"
	"- Kết quả phải đầy đủ, rõ ràng và có thể phân tích được bằng máy.\n"
)

new_prompt = ChatPromptTemplate.from_messages(
	[
		(
			"system",
			prompt_vietnam,
		),
		(
			"human",
			(
				"Tip: Make sure to answer in the correct format and do "
				"not include any explanations. "
				"Use the given format to extract information from the "
				"following input: {input}"
			),
		),
	]
)

In [1]:
from langchain.prompts import ChatPromptTemplate

# Prompt cho tác vụ nhận diện thực thể, quan hệ và thuộc tính
prompt_vietnam = (
	"# Hướng dẫn xây dựng Đồ thị Tri thức từ văn bản quy chế tiếng Việt\n"
	"## 1. Mục tiêu\n"
	"Bạn là một mô hình trích xuất thông tin chất lượng cao, được thiết kế để xây dựng đồ thị tri thức từ văn bản quy định, đặc biệt là quy chế đào tạo đại học bằng tiếng Việt.\n"
	"Hãy cố gắng trích xuất tối đa thông tin có trong văn bản **mà không thêm bất kỳ thông tin nào không được nêu rõ**.\n"
	"- **Node** là các thực thể hoặc khái niệm xuất hiện trong văn bản.\n"
	"- **Edge** là quan hệ giữa các node, phản ánh mối liên kết có trong nội dung văn bản.\n"
	"- Kết quả phải dễ hiểu, rõ ràng, có thể dùng để xây dựng hệ thống truy vấn đồ thị tri thức.\n\n"

	"## 2. Gán nhãn thực thể (Labeling Nodes)\n"
	"- Mỗi node phải có `id` là tên thực thể trích từ văn bản, và `type` là loại thực thể.\n"
	"- Nếu thực thể thuộc các loại thường gặp như: `CHƯƠNG`, `ĐIỀU`, `KHOẢN`, `CHỦ_THỂ`, `HOẠT_ĐỘNG`, `YÊU_CẦU`, `CÓ`, `LÀ`, `HỌC_LỰC`, `KHÁI_NIỆM`, ... thì hãy dùng đúng loại đó.\n"
	"- Nếu thực thể không thuộc các loại trên, HÃY TỰ ĐẶT `type` mới phù hợp bằng tiếng Việt.\n"
	"- Không được dùng số làm `id`. `id` phải là tên nguyên văn hoặc dễ hiểu từ văn bản, ví dụ: \"Chương trình đào tạo\", \"Sinh viên\".\n"
	"- Nếu trong văn bản có mô tả hoặc đặc điểm đi kèm thực thể, hãy trích xuất dưới dạng `properties`.\n"
	"- Mỗi node có thể có thêm các thuộc tính khác như `tiêu_đề`, `tên`, `định_nghĩa`, `mô_tả`, `vai_trò`, `phạm_vi`, v.v.\n"
	"- Nếu thuộc tính không nằm trong danh sách gợi ý, bạn được phép tự đặt tên phù hợp (vẫn theo chuẩn viết thường bằng tiếng Việt có dấu gạch dưới).\n\n"

	"## 3. Gán nhãn quan hệ (Relationships)\n"
	"- Các `edges` phải mô tả chính xác mối liên hệ giữa thực thể **nguồn** (`source`) và thực thể **đích** (`target`).\n"
	"- `source` là chủ thể thực hiện hoặc chủ thể quy định, và `target` là đối tượng bị tác động hoặc có liên hệ.\n"
	"- `type` của edge phải là một quan hệ rõ ràng, ngắn gọn, bằng **tiếng Việt viết HOA có dấu gạch dưới**, ví dụ: `QUY_ĐỊNH`, `BAO_GỒM`, `PHẢI_HOÀN_THÀNH`, `LIÊN_QUAN_ĐẾN`, `XẾP_LOẠI`, `THỜI_GIAN`, `LÀM`, `THUỘC`, `THAM_CHIẾU`.\n"
	"- Không dùng tên quan hệ quá cụ thể theo ngữ cảnh tạm thời, ví dụ: `BỊ_CẢNH_BÁO_2_LẦN` là không hợp lệ. Hãy dùng dạng tổng quát như `BỊ_CẢNH_BÁO`.\n"
	"- Nếu quan hệ không nằm trong danh sách gợi ý, bạn được phép tự đặt tên phù hợp (vẫn theo chuẩn viết HOA bằng tiếng Việt có dấu gạch dưới).\n"
	"- Nếu có thông tin bổ sung về quan hệ (ví dụ: yêu_cầu, điều_kiện, quy_định, điểm, số_tín_chỉ v.v.), hãy thêm vào `properties` của quan hệ.\n"
	"- Đặc biệt lưu ý: tránh nhầm lẫn `source` và `target`. Hãy đảm bảo hướng quan hệ đúng theo ngữ nghĩa.\n\n"

	"## 4. Đồng nhất thực thể (Coreference Resolution)\n"
	"- Nếu một thực thể xuất hiện nhiều lần với cách gọi khác nhau (VD: \"Trường\", \"Nhà Trường\", \"Trường Đại học Quy Nhơn\", \"trường\"), hãy dùng tên đầy đủ nhất làm ID duy nhất.\n"
	"- Không tạo trùng node cho cùng một thực thể.\n\n"

	"## 5. Tuân thủ định dạng và chính xác\n"
	"- Không giải thích, không thêm bình luận.\n"
	"- Chỉ trả về kết quả theo định dạng JSON gồm `nodes` và `edges`.\n"
	"- Kết quả phải đầy đủ, rõ ràng và có thể phân tích được bằng máy.\n"
)

new_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            prompt_vietnam,
        ),
        (
            "human",
            (
                "Tip: Make sure to answer in the correct format and do "
                "not include any explanations. "
                "Use the given format to extract information from the "
                "following input: {input}"
            ),
        ),
    ]
)


In [2]:
import os
from dotenv import load_dotenv
from langchain_deepseek import ChatDeepSeek
from langchain_core.documents import Document
from langchain_experimental.graph_transformers import LLMGraphTransformer
load_dotenv()

# Mô hình deepseek
os.environ["DEEPSEEK_API_KEY"] = str(os.getenv("DEEPSEEK_API_KEY"))
llm = ChatDeepSeek(
	model="deepseek-chat",# deepseek-reasoner, deepseek-chat
	# temperature=1,
	# max_tokens=8000,
	# timeout=None,
	# max_retries=2,
)

In [3]:
llm_transformer = LLMGraphTransformer(llm=llm, prompt=new_prompt, node_properties=True ,relationship_properties=True)

In [4]:
arr = [
'''Thời gian theo kế hoạch học tập chuẩn toàn khóa đối với hình thức đào tạo chính quy: 4,5 năm đối với các ngành đào tạo kỹ sư; 3,5 năm đối với ngành đào tạo cử nhân kỹ thuật và 4 năm đối với các ngành còn lại. Khối lượng học tập của chương trình đào tạo (không kể khối lượng các học phần Giáo dục thể chất và Giáo dục quốc phòng - an ninh):
Các ngành cấp bằng Kỹ sư: 150 tín chỉ.
Các ngành đào tạo giáo viên (ngành sư phạm): 138 tín chỉ.
Các ngành đào tạo cử nhân kỹ thuật: 120 tín chỉ.
Các ngành còn lại: 135 tín chỉ.
Khối lượng học tập của chương trình đào tạo có thể thay đổi phù hợp với yêu cầu cập nhật, điều chỉnh chương trình đào tạo nhưng phải đảm bảo quy định theo chuẩn chương trình đào tạo hiện hành của Bộ Giáo dục và Đào tạo.''']


In [161]:
# Đọc hết tất cả 14 file để chuẩn bị chuyển thành đồ thị
base_dir = os.getcwd()
arr = []
# Tạo đường dẫn đến thư mục txt
folder_path = os.path.join(base_dir, 'app', 'static', 'txt')
folder_path = 'D:/WorkSpace/python/thesis/web_graph_rag/app/static/txt'
for filename in os.listdir(folder_path):
	if filename.endswith('.txt'):
		file_path = os.path.join(folder_path, filename)
		with open(file_path, 'r', encoding='utf-8') as f:
			content = f.read()
			arr = arr + content.split('\n\n')
			# print(f"Nội dung của {filename}:\n{content}\n")

print(len(arr))

193


In [184]:
# # Đọc hết tất cả 14 file để chuẩn bị chuyển thành đồ thị
# base_dir = os.getcwd()
# # Tạo đường dẫn đến thư mục txt
# folder_path = os.path.join(base_dir, 'app', 'static', 'txt')
# folder_path = 'D:/WorkSpace/python/thesis/web_graph_rag/app/static/txt'
# for filename in os.listdir(folder_path):
# 	if filename.endswith('.txt'):
# 		file_path = os.path.join(folder_path, filename)
# 		with open(file_path, 'r', encoding='utf-8') as f:
# 			content = f.read()
# 			print(f"Nội dung của {filename}:\n")

# # print(len(arr))

In [4]:
# Đọc 1 file
base_dir = os.getcwd()
# Quy chế đào tạo trình độ đại học.txt
# Quy định về học phí của sinh viên đại học hệ chính quy.txt
# dữ liệu đánh giá.txt
arr = []
file_name = 'miss.txt'
# folder_path = os.path.join(base_dir, 'app', 'static', 'txt', file_name)
folder_path = os.path.join(base_dir, file_name)
with open(folder_path, 'r', encoding='utf-8') as f:
	content = f.read()
	arr = arr + content.split('\n\n')
	# arr.append(content)
	
len(arr)

5

In [5]:
# Tạo cấu trúc Document để vào LLMGraphTransformer xử lý
documents = []
for x in arr:
	documents.append(Document(page_content=x))

In [34]:
documents

[Document(metadata={}, page_content='Phụ lục: MỘT SỐ NỘI DUNG VI PHẠM VÀ KHUNG XỬ LÝ KHI VI PHẠM LUẬT SINH VIÊN\n13. Tàng trữ, lưu hành, truy cập, sử dụng sản phẩm văn hóa đồi trụy hoặc tham gia các hoạt động mê tín dị đoan, hoạt động tôn giáo trái phép:  \nLần 1: Khiển trách  \nLần 2: Cảnh cáo  \nLần 3: Đình chỉ có thời hạn  \nLần 4: Buộc thôi học và giao cho cơ quan chức năng xử lý theo quy định pháp luật  \n14. Buôn bán, vận chuyển, tàng trữ, lôi kéo người khác sử dụng ma túy:  \nXử lý nghiêm, giao cho cơ quan chức năng xử lý theo quy định pháp luật  \n15. Sử dụng ma túy:  \nBuộc thôi học và xử lý theo quy định về sinh viên liên quan đến ma túy  \n16. Chứa chấp, môi giới mại dâm:  \nBuộc thôi học và giao cho cơ quan chức năng xử lý theo quy định pháp luật  \n17. Hoạt động mại dâm:  \nLần 1: Khiển trách  \nLần 2: Cảnh cáo  \nLần 3: Đình chỉ có thời hạn  \nLần 4: Buộc thôi học và xử lý theo quy định pháp luật  \n18. Lấy cắp tài sản, chứa chấp, tiêu thụ tài sản do lấy cắp mà có:  \nTùy

In [6]:
# Chuyển các document thành các thực thể và quan hệ
graph_documents = llm_transformer.convert_to_graph_documents(documents=documents)

In [183]:
graph_full_temperature_0 = graph_documents

In [37]:
# Một các hiển thị dễ quan sát hơn
for x in graph_documents:
	for y in x.nodes:
		print(f"{y.type} - {y.id} - {y.properties}")
	print()
	for y in x.relationships:
		print(f"({y.source.type}: {y.source.id}) - ({y.type}:{y.properties}) - ({y.target.type}:{y.target.id})")
	
	print('-'*40)
	print()


----------------------------------------



In [8]:
graph_documents[0]

GraphDocument(nodes=[Node(id='Hình Thức Đào Tạo Chính Quy', type='Hình_thức_đào_tạo', properties={}), Node(id='Ngành Đào Tạo Kỹ Sư', type='Ngành_đào_tạo', properties={}), Node(id='Ngành Đào Tạo Cử Nhân Kỹ Thuật', type='Ngành_đào_tạo', properties={}), Node(id='Các Ngành Còn Lại', type='Ngành_đào_tạo', properties={}), Node(id='Chương Trình Đào Tạo', type='Chương_trình_đào_tạo', properties={}), Node(id='Khối Lượng Học Tập', type='Khối_lượng_học_tập', properties={}), Node(id='Học Phần Giáo Dục Thể Chất', type='Học_phần', properties={}), Node(id='Học Phần Giáo Dục Quốc Phòng - An Ninh', type='Học_phần', properties={}), Node(id='Bộ Giáo Dục Và Đào Tạo', type='Cơ_quan', properties={}), Node(id='Ngành Cấp Bằng Kỹ Sư', type='Ngành_đào_tạo', properties={}), Node(id='Ngành Đào Tạo Giáo Viên', type='Ngành_đào_tạo', properties={})], relationships=[Relationship(source=Node(id='Hình Thức Đào Tạo Chính Quy', type='Hình_thức_đào_tạo', properties={}), target=Node(id='Ngành Đào Tạo Kỹ Sư', type='Ngành_đà

In [ ]:
GraphDocument(
  nodes=[
    Node(
			id='Hình Thức Đào Tạo Chính Quy', 
    	type='Hình_thức_đào_tạo', properties={}
    ),
    Node(
      id='Ngành Đào Tạo Kỹ Sư', 
      type='Ngành_đào_tạo', properties={}
    ),
    ...
  ],
  relationships=[
    Relationship(
      source=Node(
        id='Hình Thức Đào Tạo Chính Quy', 
        type='Hình_thức_đào_tạo', properties={}
      ),
      target=Node(
        id='Ngành Đào Tạo Kỹ Sư', 
        type='Ngành_đào_tạo', properties={}
      ),
      type='THỜI_GIAN',
      properties={'thời_gian': '4,5 năm'}
    ),
    Relationship(
      source=Node(
        id='Hình Thức Đào Tạo Chính Quy', 
        type='Hình_thức_đào_tạo', properties={}
      ),
      target=Node(
        id='Ngành Đào Tạo Cử Nhân Kỹ Thuật', 
        type='Ngành_đào_tạo', properties={}),
      type='THỜI_GIAN',
      properties={'thời_gian': '3,5 năm'}
    ),
    ...
	]
)
    Relationship(
      source=Node(id='Hình Thức Đào Tạo Chính Quy', type='Hình_thức_đào_tạo', properties={}),
      target=Node(id='Các Ngành Còn Lại', type='Ngành_đào_tạo', properties={}),
      type='THỜI_GIAN',
      properties={'thời_gian': '4 năm'}
    ),
    Relationship(
      source=Node(id='Chương Trình Đào Tạo', type='Chương_trình_đào_tạo', properties={}),
      target=Node(id='Khối Lượng Học Tập', type='Khối_lượng_học_tập', properties={}),
      type='BAO_GỒM',
      properties={}
    ),
    Relationship(
      source=Node(id='Khối Lượng Học Tập', type='Khối_lượng_học_tập', properties={}),
      target=Node(id='Học Phần Giáo Dục Thể Chất', type='Học_phần', properties={}),
      type='KHÔNG_BAO_GỒM',
      properties={}
    ),
    Relationship(
      source=Node(id='Khối Lượng Học Tập', type='Khối_lượng_học_tập', properties={}),
      target=Node(id='Học Phần Giáo Dục Quốc Phòng - An Ninh', type='Học_phần', properties={}),
      type='KHÔNG_BAO_GỒM',
      properties={}
    ),
    Relationship(
      source=Node(id='Ngành Cấp Bằng Kỹ Sư', type='Ngành_đào_tạo', properties={}),
      target=Node(id='Khối Lượng Học Tập', type='Khối_lượng_học_tập', properties={}),
      type='YÊU_CẦU',
      properties={'số_tín_chỉ': '150'}
    ),
    Relationship(
      source=Node(id='Ngành Đào Tạo Giáo Viên', type='Ngành_đào_tạo', properties={}),
      target=Node(id='Khối Lượng Học Tập', type='Khối_lượng_học_tập', properties={}),
      type='YÊU_CẦU',
      properties={'số_tín_chỉ': '138'}
    ),
    Relationship(
      source=Node(id='Ngành Đào Tạo Cử Nhân Kỹ Thuật', type='Ngành_đào_tạo', properties={}),
      target=Node(id='Khối Lượng Học Tập', type='Khối_lượng_học_tập', properties={}),
      type='YÊU_CẦU',
      properties={'số_tín_chỉ': '120'}
    ),
    Relationship(
      source=Node(id='Các Ngành Còn Lại', type='Ngành_đào_tạo', properties={}),
      target=Node(id='Khối Lượng Học Tập', type='Khối_lượng_học_tập', properties={}),
      type='YÊU_CẦU',
      properties={'số_tín_chỉ': '135'}
    ),
    Relationship(
      source=Node(id='Chương Trình Đào Tạo', type='Chương_trình_đào_tạo', properties={}),
      target=Node(id='Bộ Giáo Dục Và Đào Tạo', type='Cơ_quan', properties={}),
      type='TUÂN_THỦ',
      properties={}
    )
  ],
  source=Document(
    metadata={},
    page_content='''
Thời gian theo kế hoạch học tập chuẩn toàn khóa đối với hình thức đào tạo chính quy: 4,5 năm đối với các ngành đào tạo kỹ sư; 3,5 năm đối với ngành đào tạo cử nhân kỹ thuật và 4 năm đối với các ngành còn lại. Khối lượng học tập của chương trình đào tạo (không kể khối lượng các học phần Giáo dục thể chất và Giáo dục quốc phòng - an ninh):
Các ngành cấp bằng Kỹ sư: 150 tín chỉ.
Các ngành đào tạo giáo viên (ngành sư phạm): 138 tín chỉ.
Các ngành đào tạo cử nhân kỹ thuật: 120 tín chỉ.
Các ngành còn lại: 135 tín chỉ.
Khối lượng học tập của chương trình đào tạo có thể thay đổi phù hợp với yêu cầu cập nhật, điều chỉnh chương trình đào tạo nhưng phải đảm bảo quy định theo chuẩn chương trình đào tạo hiện hành của Bộ Giáo dục và Đào Tạo.
'''
  )
)


In [13]:
import json

graph_data = {
    "nodes": [node.model_dump() for node in graph_documents[0].nodes],
    "edges": [rel.model_dump() for rel in graph_documents[0].relationships],
}

with open("D:/WorkSpace/python/thesis/web_graph_rag/graph_output.json", "w", encoding="utf-8") as f:
    json.dump(graph_data, f, ensure_ascii=False, indent=2)


In [165]:
# Mở file để ghi output chung và file cho các graph_documents không có nodes và relationships
path = 'D:/WorkSpace/python/thesis/web_graph_rag/'
with open(path + 'output_all.txt', 'w', encoding='utf-8') as f_all, open(path + 'no_nodes_or_relationships.txt', 'w', encoding='utf-8') as f_no_nodes:
	for x in graph_documents:
		has_nodes_or_relationships = False

		# Kiểm tra và ghi nodes vào file
		for y in x.nodes:
			f_all.write(f"{y.type} - {y.id} - {y.properties}\n")
			has_nodes_or_relationships = True
		
		# Kiểm tra và ghi relationships vào file
		for y in x.relationships:
			f_all.write(f"({y.source.type}: {y.source.id}) - ({y.type}:{y.properties}) - ({y.target.type}:{y.target.id})\n")
			has_nodes_or_relationships = True

		# Nếu không có nodes và relationships, ghi thông tin vào file riêng biệt
		if not has_nodes_or_relationships:
			f_no_nodes.write(f"GraphDocument không có nodes và relationships:\n{x.source.page_content}\n")
			f_no_nodes.write('-' * 40 + '\n')
		
		# Thêm một dòng phân cách
		f_all.write('-' * 40 + '\n')

print("Đã ghi dữ liệu vào các file.")


Đã ghi dữ liệu vào các file.


In [14]:
# Hàm để chuyển các thực thể và quan hệ vào neo4j
def insert_nodes_and_relationships(driver, nodes, relationships):
	with driver.session() as session:
		# Chèn nodes
		for node in nodes:
			x = str(node.type).replace(',', '')
			x = x.replace(' ', '_')

			cypher_query = f"""
				MERGE (n:{x} {{id: $id}})
				SET n += $properties
			"""
			session.run(
				cypher_query,
				id=node.id,
				properties=node.properties
			)

		# Chèn relationships
		for relationship in relationships:
			x = str(relationship.source.type).replace(',', '')
			x = x.replace(' ', '_')

			y = str(relationship.target.type).replace(',', '')
			y = y.replace(' ', '_')

			z = str(relationship.type).replace(',', '')
			z = z.replace(' ', '_')

			cypher_query = f"""
				MATCH (a:{x} {{id: $source_id}}), 
							(b:{y} {{id: $target_id}})
				MERGE (a)-[r:{z}]->(b)
				SET r += $properties
			"""	
			session.run(
				cypher_query,
				source_id=relationship.source.id,
				target_id=relationship.target.id,
				properties=relationship.properties
			)

In [15]:
def insert_nodes_and_relationships(driver, nodes, relationships):
	with driver.session() as session:
		# Chèn nodes
		for node in nodes:
			# Thay thế dấu ',' và khoảng trắng trong tên loại node thành '_'
			node_type = str(node.type).replace(',', '').replace(' ', '_').replace('%', '_PHẦN_TRĂM')
			
			# Câu lệnh Cypher để MERGE node
			cypher_query = f"""
				MERGE (n:{node_type} {{id: $id}})
				SET n += $properties
			"""
			session.run(
				cypher_query,
				id=node.id,
				properties=node.properties
			)

		# Chèn relationships
		for relationship in relationships:
			# Thay thế dấu ',' và khoảng trắng trong tên loại node thành '_'
			source_type = str(relationship.source.type).replace(',', '').replace(' ', '_').replace('%', '_PHẦN_TRĂM')
			target_type = str(relationship.target.type).replace(',', '').replace(' ', '_').replace('%', '_PHẦN_TRĂM')
			relationship_type = str(relationship.type).replace(',', '').replace(' ', '_').replace('%', '_PHẦN_TRĂM')
			
			# Câu lệnh Cypher để MERGE relationship
			cypher_query = f"""
				MATCH (a:{source_type} {{id: $source_id}}), 
							(b:{target_type} {{id: $target_id}})
				MERGE (a)-[r:{relationship_type}]->(b)
				SET r += $properties
			"""
			session.run(
				cypher_query,
				source_id=relationship.source.id,
				target_id=relationship.target.id,
				properties=relationship.properties
			)


In [16]:
# Hàm tạo connect đến neo4j
from neo4j import GraphDatabase

def connect_to_neo4j(uri, user, password, db):
	driver = GraphDatabase.driver(uri, auth=(user, password), database=db)
	return driver

In [17]:
# Kết nối đến Neo4j
uri = "bolt://localhost:7687"
user = "neo4j"
password = "adminadmin"
db = 'vidu'
driver = connect_to_neo4j(uri, user, password, db)

In [18]:
for x in graph_documents:
	insert_nodes_and_relationships(driver, x.nodes, x.relationships)


In [177]:
graph_documents[-1]

GraphDocument(nodes=[Node(id='Chương V', type='Chương', properties={'tiêu_đề': 'TỔ CHỨC THỰC HIỆN'}), Node(id='Điều 16', type='Điều', properties={'tiêu_đề': 'Tổ chức thực hiện'}), Node(id='Hiệu Trưởng Nhà Trường', type='Chủ_thể', properties={}), Node(id='Văn Bản Hướng Dẫn Chi Tiết', type='Văn_bản', properties={}), Node(id='Phiếu Đánh Giá Kết Quả Rèn Luyện Của Sinh Viên', type='Văn_bản', properties={}), Node(id='Phòng', type='Bộ_phận', properties={}), Node(id='Ban', type='Bộ_phận', properties={}), Node(id='Khoa', type='Bộ_phận', properties={}), Node(id='Lớp', type='Bộ_phận', properties={}), Node(id='Giáo Viên Chủ Nhiệm', type='Chủ_thể', properties={}), Node(id='Cố Vấn Học Tập', type='Chủ_thể', properties={}), Node(id='Người Học', type='Chủ_thể', properties={}), Node(id='Đơn Vị Chức Năng', type='Bộ_phận', properties={}), Node(id='Bộ Giáo Dục Và Đào Tạo', type='Chủ_thể', properties={})], relationships=[Relationship(source=Node(id='Chương V', type='Chương', properties={}), target=Node(id='

In [ ]:
driver.close()


In [51]:
# Prompt cho tác vụ tìm kiếm(cũ)
CYPHER_GENERATION_TEMPLATE = """
Task: Sinh truy vấn Cypher để truy xuất dữ liệu từ đồ thị tri thức về quy chế đào tạo đại học.

Instructions:
- Phân tích kỹ câu hỏi và trích xuất các thành phần quan trọng như thực thể (node), quan hệ (edge) và thuộc tính (property).
- Chỉ sử dụng các loại node, quan hệ và thuộc tính có trong schema bên dưới. Không thêm bất kỳ phần tử nào không có trong schema.
- Các nhãn node được đặt bằng tiếng Việt (ví dụ: :Chương_trình, :Chủ_thể, :Học, :Quy_định...).
- Các nhãn quan hệ được đặt bằng tiếng Việt (ví dụ: :ÁP_DỤNG, :HỌC, :QUY_ĐỊNH...).
- Truy vấn phải trả lời chính xác câu hỏi dựa trên cấu trúc đồ thị.
- Chỉ trả về truy vấn Cypher. Không viết giải thích, ghi chú, hoặc định nghĩa gì thêm.

Schema đồ thị:
{schema}

Ví dụ truy vấn:

# Các phương thức đóng học phí bao gồm các phương thức nào?
# Cách 1
MATCH (n:`Hoạt_động`)-[]->(m)
WHERE n.id contains 'Phương Thức Đóng Học Phí'
return n, m
# Cách 2
MATCH (n:`Phương_thức`)-[:`ÁP_DỤNG_CHO`]->(m)
WHERE m.id contains 'Sinh Viên'
return n.id

# Học phần bắt buộc là gì?
MATCH (n: Loại_học_phần)
WHERE n.id contains 'Học Phần Bắt Buộc'
return n.`định_nghĩa`

# Khi nào sinh viên bị buộc thôi học?
MATCH (n: Chủ_thể)-[:`CÓ_HÀNH_VI`]->(m)-[:`BỊ_XỬ_LÝ`]->(x)
WHERE n.id contains 'Sinh Viên' 
AND x.id contains 'Buộc Thôi Học'
return n, m, x

# Thời hạn đóng học phí là khi nào?
MATCH (n:`Chủ_thể`)-[r1:`PHẢI_TUÂN_THỦ`]->(m)-[r2]->(x)
where n.id contains 'Sinh Viên' 
and m.id contains 'Thời Hạn Đóng Học Phí'
RETURN n, r1, m, r2, x 

The question is:
{question}
"""


In [57]:

CYPHER_GENERATION_TEMPLATE = """
Task: Sinh truy vấn Cypher để truy xuất dữ liệu từ đồ thị tri thức.

Instructions:
- Phân tích kỹ câu hỏi và xác định các thành phần quan trọng như thực thể (node), quan hệ (edge), và thuộc tính (property).
- Chỉ sử dụng các loại node, quan hệ và thuộc tính có trong schema bên dưới. Không thêm bất kỳ phần tử nào không có trong schema.
- Các nhãn node được đặt bằng tiếng Việt (ví dụ: :Chương_trình, :Chủ_thể, :Học, :Quy_định...).
- Các loại quan hệ cũng được đặt bằng tiếng Việt (ví dụ: :ÁP_DỤNG, :HỌC, :QUY_ĐỊNH...).
- Mọi truy vấn **phải trả về đầy đủ tất cả thuộc tính** của node và relationship bằng `properties(...)`. Nếu cần, sử dụng `type(...)` để lấy tên quan hệ.
- **Chỉ được sử dụng phép so sánh `CONTAINS` trong mệnh đề WHERE. Không được sử dụng phép gán bằng (`=`).**
- Khi cần truy vấn nhiều mẫu (nhiều thực thể và quan hệ khác nhau), hãy dùng nhiều câu `OPTIONAL MATCH` tách riêng, sau đó gom kết quả lại bằng `collect(...)` và chỉ viết một câu `RETURN` duy nhất ở cuối.
- Không viết nhiều câu `OPTIONAL MATCH ... RETURN` liên tiếp. Luôn gom kết quả lại để trả về một lần duy nhất.
- Không viết nhiều truy vấn MATCH...RETURN liên tiếp mà không tách bằng dòng trắng hoặc dùng `UNION`.
- Không sử dụng `UNION` nếu có thể gộp các mẫu bằng `OPTIONAL MATCH`.
- Không viết lời giải thích, ghi chú hoặc định nghĩa. Chỉ trả về truy vấn Cypher.

Schema đồ thị:
{schema}

Ví dụ truy vấn:

# Các phương thức đóng học phí bao gồm các phương thức nào?
OPTIONAL MATCH (n)-[r]->(m)
WHERE n.id CONTAINS 'Phương Thức Đóng Học Phí'
RETURN 
  properties(n) AS n_properties, 
  type(r) AS r_type, 
  properties(r) AS r_properties, 
  properties(m) AS m_properties

# Học phần bắt buộc là gì?
OPTIONAL MATCH (n)
WHERE n.id CONTAINS 'Học Phần Bắt Buộc'
RETURN properties(n) AS n_properties

# Khi nào sinh viên bị buộc thôi học?
OPTIONAL MATCH (n)-[r1:`BỊ_BUỘC_THÔI_HỌC`]->(m)
WHERE n.id CONTAINS 'Sinh Viên'
RETURN 
  properties(n) AS n_properties, 
  type(r1) AS r1_type, 
  properties(r1) AS r1_properties,
  properties(m) AS m_properties

# Khối lượng học tập của các ngành là bao nhiêu?
OPTIONAL MATCH (n)
WHERE n.id CONTAINS 'Khối Lượng Học Tập'

OPTIONAL MATCH (n2:Ngành_đào_tạo)-[r]->(m)
WHERE m.id CONTAINS 'Khối Lượng Học Tập'

RETURN 
  collect(DISTINCT properties(n)) AS khái_niệm,
  collect(DISTINCT properties(n2)) AS ngành_đào_tạo,
  collect(DISTINCT type(r)) AS loại_quan_hệ,
  collect(DISTINCT properties(r)) AS thuộc_tính_quan_hệ,
  collect(DISTINCT properties(m)) AS khái_niệm_liên_quan

The question is:
{question}
"""


In [1]:
CYPHER_GENERATION_TEMPLATE = """
Task: Sinh truy vấn Cypher để truy xuất dữ liệu từ đồ thị tri thức.

Instructions:
- Phân tích kỹ câu hỏi và xác định các thành phần quan trọng như thực thể (node), quan hệ (edge), và thuộc tính (property).
- Chỉ sử dụng các loại node, quan hệ và thuộc tính có trong schema bên dưới. Không thêm bất kỳ phần tử nào không có trong schema.
- Không sử dụng nhãn node trong MATCH (chỉ viết OPTIONAL MATCH (n), không OPTIONAL MATCH (n:Label)).
- Các loại quan hệ được đặt bằng tiếng Việt (ví dụ: :ÁP_DỤNG, :HỌC, :QUY_ĐỊNH...).
- Tất cả truy vấn đều phải dùng `OPTIONAL MATCH`. Không sử dụng `MATCH`.
- Mọi truy vấn **phải trả về đầy đủ tất cả thuộc tính** của node và relationship bằng `properties(...)`. Nếu cần, sử dụng `type(...)` để lấy tên quan hệ.
- **Chỉ được sử dụng phép so sánh `CONTAINS` trong mệnh đề WHERE, và luôn sử dụng `toLower(...)` ở cả hai vế.**
  Ví dụ: `toLower(n.id) CONTAINS toLower('từ khóa')`
- Khi cần truy vấn nhiều mẫu (nhiều thực thể và quan hệ khác nhau), hãy dùng nhiều câu `OPTIONAL MATCH` tách riêng, sau đó gom kết quả lại bằng `collect(...)` và chỉ viết một câu `RETURN` duy nhất ở cuối.
- Không sử dụng `UNION` trong bất kỳ trường hợp nào.
- Không viết nhiều câu `OPTIONAL MATCH...RETURN` liên tiếp. Luôn gom kết quả lại để trả về một lần duy nhất bằng một câu `RETURN`.
- Không viết lời giải thích, ghi chú hoặc định nghĩa. Chỉ trả về truy vấn Cypher.

Schema đồ thị:
{schema}

Ví dụ truy vấn:

# Các phương thức đóng học phí bao gồm các phương thức nào?
OPTIONAL MATCH (n)-[r]->(m)
WHERE toLower(n.id) CONTAINS toLower('Phương Thức Đóng Học Phí')

RETURN 
  collect(DISTINCT properties(n)) AS n_properties, 
  collect(DISTINCT type(r)) AS r_type, 
  collect(DISTINCT properties(r)) AS r_properties, 
  collect(DISTINCT properties(m)) AS m_properties

# Học phần bắt buộc là gì?
OPTIONAL MATCH (n)
WHERE toLower(n.id) CONTAINS toLower('Học Phần Bắt Buộc')

RETURN collect(DISTINCT properties(n)) AS n_properties

# Khi nào sinh viên bị buộc thôi học?
OPTIONAL MATCH (n)-[r1]->(m)
WHERE toLower(n.id) CONTAINS toLower('Sinh Viên') AND type(r1) = 'BỊ_BUỘC_THÔI_HỌC'

RETURN 
  collect(DISTINCT properties(n)) AS n_properties, 
  collect(DISTINCT type(r1)) AS r1_type, 
  collect(DISTINCT properties(r1)) AS r1_properties,
  collect(DISTINCT properties(m)) AS m_properties

# Khối lượng học tập của các ngành là bao nhiêu?
OPTIONAL MATCH (n)
WHERE toLower(n.id) CONTAINS toLower('Khối Lượng Học Tập')

OPTIONAL MATCH (n2)-[r]->(m)
WHERE toLower(m.id) CONTAINS toLower('Khối Lượng Học Tập')

RETURN 
  collect(DISTINCT properties(n)) AS khái_niệm,
  collect(DISTINCT properties(n2)) AS ngành_đào_tạo,
  collect(DISTINCT type(r)) AS loại_quan_hệ,
  collect(DISTINCT properties(r)) AS thuộc_tính_quan_hệ,
  collect(DISTINCT properties(m)) AS khái_niệm_liên_quan

The question is:
{question}
"""


In [1]:
CYPHER_GENERATION_TEMPLATE = """
Task: Sinh truy vấn Cypher để truy xuất dữ liệu từ đồ thị tri thức.

Instructions:
- Phân tích kỹ câu hỏi và xác định các thành phần quan trọng như thực thể (node), quan hệ (edge), và thuộc tính (property).
- Chỉ sử dụng các loại node, quan hệ và thuộc tính có trong schema bên dưới. Không thêm bất kỳ phần tử nào không có trong schema.
- Không sử dụng nhãn node trong MATCH (chỉ viết OPTIONAL MATCH (n), không OPTIONAL MATCH (n:Label)).
- Các loại quan hệ được đặt bằng tiếng Việt (ví dụ: :ÁP_DỤNG, :HỌC, :QUY_ĐỊNH...).
- Tất cả truy vấn đều phải dùng `OPTIONAL MATCH`. Không sử dụng `MATCH`.

- Mọi truy vấn **phải trả về đầy đủ tất cả thuộc tính** của node và relationship bằng `properties(...)`. Nếu cần, sử dụng `type(...)` để lấy tên quan hệ.
- **Chỉ được sử dụng phép so sánh `CONTAINS` trong mệnh đề WHERE, và luôn sử dụng `toLower(...)` ở cả hai vế.**
  Ví dụ: `toLower(n.id) CONTAINS toLower('từ khóa')`

- Nếu câu hỏi chỉ yêu cầu thông tin về một khái niệm hoặc thực thể đơn lẻ → sử dụng mẫu:
  `OPTIONAL MATCH (n) WHERE ...`
- Nếu câu hỏi yêu cầu mối liên hệ hoặc dữ kiện liên quan giữa các khái niệm → sử dụng mẫu:
  `OPTIONAL MATCH (n)-[r]-(m) WHERE ...`

- Khi cần truy vấn nhiều mẫu (nhiều thực thể và quan hệ khác nhau), hãy dùng nhiều câu `OPTIONAL MATCH` tách riêng, sau đó gom kết quả lại bằng `collect(...)` và chỉ viết một câu `RETURN` duy nhất ở cuối.
- Không sử dụng `UNION` trong bất kỳ trường hợp nào.
- Không viết nhiều câu `OPTIONAL MATCH...RETURN` liên tiếp. Luôn gom kết quả lại để trả về một lần duy nhất bằng một câu `RETURN`.
- Không viết lời giải thích, ghi chú hoặc định nghĩa. Chỉ trả về truy vấn Cypher.

Schema đồ thị:
{schema}

Ví dụ truy vấn:

# Các phương thức đóng học phí bao gồm các phương thức nào?
OPTIONAL MATCH (n)-[r]->(m)
WHERE toLower(n.id) CONTAINS toLower('Phương Thức Đóng Học Phí')

RETURN 
  collect(DISTINCT properties(n)) AS n_properties, 
  collect(DISTINCT type(r)) AS r_type, 
  collect(DISTINCT properties(r)) AS r_properties, 
  collect(DISTINCT properties(m)) AS m_properties

# Học phần bắt buộc là gì?
OPTIONAL MATCH (n)
WHERE toLower(n.id) CONTAINS toLower('Học Phần Bắt Buộc')

RETURN collect(DISTINCT properties(n)) AS n_properties

# Khi nào sinh viên bị buộc thôi học?
OPTIONAL MATCH (n)-[r1]->(m)
WHERE toLower(n.id) CONTAINS toLower('Sinh Viên') AND type(r1) = 'BỊ_BUỘC_THÔI_HỌC'

RETURN 
  collect(DISTINCT properties(n)) AS n_properties, 
  collect(DISTINCT type(r1)) AS r1_type, 
  collect(DISTINCT properties(r1)) AS r1_properties,
  collect(DISTINCT properties(m)) AS m_properties

# Khối lượng học tập của các ngành là bao nhiêu?
OPTIONAL MATCH (n)
WHERE toLower(n.id) CONTAINS toLower('Khối Lượng Học Tập')

OPTIONAL MATCH (n2)-[r]->(m)
WHERE toLower(m.id) CONTAINS toLower('Khối Lượng Học Tập')

RETURN 
  collect(DISTINCT properties(n)) AS khái_niệm,
  collect(DISTINCT properties(n2)) AS ngành_đào_tạo,
  collect(DISTINCT type(r)) AS loại_quan_hệ,
  collect(DISTINCT properties(r)) AS thuộc_tính_quan_hệ,
  collect(DISTINCT properties(m)) AS khái_niệm_liên_quan

# Những quy định liên quan đến điều kiện tốt nghiệp là gì?
OPTIONAL MATCH (n)-[r]-(m)
WHERE toLower(n.id) CONTAINS toLower('Điều Kiện Tốt Nghiệp')

RETURN 
  collect(DISTINCT properties(n)) AS n_properties, 
  collect(DISTINCT type(r)) AS r_type, 
  collect(DISTINCT properties(r)) AS r_properties, 
  collect(DISTINCT properties(m)) AS m_properties

The question is:
{question}
"""


In [2]:
from langchain.prompts.prompt import PromptTemplate

CYPHER_GENERATION_PROMPT = PromptTemplate(
	input_variables=["schema", "question"],
	template=CYPHER_GENERATION_TEMPLATE
)

In [3]:
import os
from dotenv import load_dotenv
from langchain_deepseek import ChatDeepSeek
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
load_dotenv()

graph = Neo4jGraph(url="bolt://localhost:7687", username="neo4j", password="adminadmin", database="quydinh-quyche-full-temperature-0", enhanced_schema=True)

# print(graph.schema)

# Mô hình deepseek
os.environ["DEEPSEEK_API_KEY"] = str(os.getenv("DEEPSEEK_API_KEY"))
llm = ChatDeepSeek(
	model="deepseek-chat",# deepseek-reasoner, deepseek-chat
	temperature=0,
	max_tokens=8000,
	timeout=None,
	max_retries=2,
)

chain = GraphCypherQAChain.from_llm(
	llm,
	graph=graph,
	verbose=True,
	top_k=5,
	allow_dangerous_requests=True,
	validate_cypher=True,
	return_intermediate_steps=True,
	cypher_prompt=CYPHER_GENERATION_PROMPT,
)

In [4]:
# Câu hỏi đầu vào
question = "Giới thiệu thư viện trường đại học quy nhơn?"

# Tạo câu lệnh Cypher từ câu hỏi
args = {"question": question, "schema": chain.graph_schema}  # Cung cấp schema của đồ thị
generated_cypher = chain.cypher_generation_chain.invoke(args)
print("Generated Cypher Query:", generated_cypher)


Generated Cypher Query: OPTIONAL MATCH (n)
WHERE toLower(n.id) CONTAINS toLower('Thư Viện Qnu')

OPTIONAL MATCH (n)-[r1]-(m1)
WHERE toLower(n.id) CONTAINS toLower('Thư Viện Qnu')

OPTIONAL MATCH (n)-[r2]-(m2)
WHERE toLower(m2.id) CONTAINS toLower('Trường Đại Học Quy Nhơn')

RETURN 
  collect(DISTINCT properties(n)) AS thuvien_properties,
  collect(DISTINCT type(r1)) AS relationship_types,
  collect(DISTINCT properties(r1)) AS relationship_properties,
  collect(DISTINCT properties(m1)) AS related_entities,
  collect(DISTINCT properties(m2)) AS truong_properties


In [5]:
# Kiểm tra và sửa câu lệnh Cypher (nếu có)
if chain.cypher_query_corrector:
    generated_cypher = chain.cypher_query_corrector(generated_cypher)
    print("Corrected Cypher Query:", generated_cypher)


Corrected Cypher Query: OPTIONAL MATCH (n)
WHERE toLower(n.id) CONTAINS toLower('Thư Viện Qnu')

OPTIONAL MATCH (n)-[r1]-(m1)
WHERE toLower(n.id) CONTAINS toLower('Thư Viện Qnu')

OPTIONAL MATCH (n)-[r2]-(m2)
WHERE toLower(m2.id) CONTAINS toLower('Trường Đại Học Quy Nhơn')

RETURN 
  collect(DISTINCT properties(n)) AS thuvien_properties,
  collect(DISTINCT type(r1)) AS relationship_types,
  collect(DISTINCT properties(r1)) AS relationship_properties,
  collect(DISTINCT properties(m1)) AS related_entities,
  collect(DISTINCT properties(m2)) AS truong_properties


In [6]:
# Thực thi câu lệnh Cypher trên cơ sở dữ liệu đồ thị
context = chain.graph.query(generated_cypher)[: chain.top_k]
print("Context (Database Results):", context)


Context (Database Results): [{'thuvien_properties': [{'id': 'Thư Viện Qnu'}], 'relationship_types': ['TRUY_CẬP', 'HỢP_TÁC', 'MUA_QUYỀN_KHAI_THÁC', 'KHAI_THÁC', 'THAM_GIA'], 'relationship_properties': [{'hệThống': 'Hệ thống máy tính của Thư viện', 'phươngThức': 'Trực tiếp'}, {}], 'related_entities': [{'id': 'Bạn Đọc'}, {'id': 'Trung Tâm Học Liệu Trường Đại Học Cần Thơ'}, {'id': 'Mathscinet'}, {'id': 'Thư Viện Pháp Luật Trực Tuyến'}, {'id': 'Kqnc'}, {'id': 'Std'}, {'id': 'Proquest Central'}, {'id': 'Liên Hiệp Thư Viện Việt Nam'}], 'truong_properties': []}]


In [7]:
# Trả lời câu hỏi sử dụng ngữ cảnh và chuỗi QA
qa_input = {"question": question, "context": context}
final_result = chain.qa_chain.invoke(qa_input)
print("Final Answer:", final_result)


Final Answer: Thư viện trường Đại học Quy Nhơn có tên là Thư Viện Qnu. Thư viện này có các mối quan hệ như TRUY_CẬP, HỢP_TÁC, MUA_QUYỀN_KHAI_THÁC, KHAI_THÁC, và THAM_GIA với các đối tác như Bạn Đọc, Trung Tâm Học Liệu Trường Đại Học Cần Thơ, Mathscinet, Thư Viện Pháp Luật Trực Tuyến, Kqnc, Std, Proquest Central, và Liên Hiệp Thư Viện Việt Nam. Hệ thống máy tính của Thư viện hỗ trợ phương thức truy cập trực tiếp.


In [8]:
import os
import pandas as pd
from tqdm import tqdm

# Đọc dữ liệu
df = pd.read_excel(os.path.join(os.getcwd(), 'danh_gia', 'Danh sách câu hỏi đánh giá.xlsx'))
list_query_one_node = df['Truy vấn 1 thực thể'].dropna().tolist()
list_query_two_node = df['Câu hỏi 2 thực thể 1 quan hệ'].dropna().tolist()

# Hàm xử lý từng nhóm câu hỏi
def process_queries(queries):
    results = []
    for query in tqdm(queries):
        try:
            result = chain.invoke({"query": query})
            cypher = result.get("intermediate_steps", [{}])[0].get("query", "")
            answer = result.get("result", "")
            results.append({
                "Câu hỏi": query,
                "Câu lệnh Cypher": cypher,
                "Câu trả lời": answer
            })
        except Exception as e:
            results.append({
                "Câu hỏi": query,
                "Câu lệnh Cypher": "LỖI",
                "Câu trả lời": f"Lỗi: {e}"
            })
    return pd.DataFrame(results)

# Xử lý từng danh sách
df_one_node = process_queries(list_query_one_node)
df_two_node = process_queries(list_query_two_node)

# Ghi ra file Excel
output_path = os.path.join(os.getcwd(), 'danh_gia', "output_results.xlsx")
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    df_one_node.to_excel(writer, sheet_name="Truy vấn 1 thực thể", index=False)
    df_two_node.to_excel(writer, sheet_name="Truy vấn 2 thực thể", index=False)

print("✅ Đã lưu kết quả vào:", output_path)

In [9]:
result = chain.invoke({"query": "Giới thiệu thư viện trường đại học quy nhơn?"})

print(f"Final answer: {result['result']}")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
OPTIONAL MATCH (n)
WHERE toLower(n.id) CONTAINS toLower('Thư Viện Qnu')

OPTIONAL MATCH (n)-[r1]-(m1)
WHERE toLower(n.id) CONTAINS toLower('Thư Viện Qnu')

OPTIONAL MATCH (n2)-[r2]-(m2)
WHERE toLower(m2.id) CONTAINS toLower('Thư Viện Qnu')

RETURN 
  collect(DISTINCT properties(n)) AS thuvien_properties,
  collect(DISTINCT type(r1)) AS r1_type,
  collect(DISTINCT properties(r1)) AS r1_properties,
  collect(DISTINCT properties(m1)) AS m1_properties,
  collect(DISTINCT properties(n2)) AS n2_properties,
  collect(DISTINCT type(r2)) AS r2_type,
  collect(DISTINCT properties(r2)) AS r2_properties,
  collect(DISTINCT properties(m2)) AS m2_properties
Full Context:
[{'thuvien_properties': [{'id': 'Thư Viện Qnu'}], 'r1_type': ['TRUY_CẬP', 'HỢP_TÁC', 'MUA_QUYỀN_KHAI_THÁC', 'KHAI_THÁC', 'THAM_GIA'], 'r1_properties': [{'hệThống': 'Hệ thống máy tính của Thư viện', 'phươngThức': 'Trực tiếp'}, {}], 'm1_properties': [{'id': 'Bạn Đọc'}, {'i

In [56]:
result

NameError: name 'result' is not defined

In [18]:
# Cách hiển thị Final answer dễ quan sát hơn
import textwrap

def prettyCypherChain(question: str) -> str:
	response = chain.run(question)
	print(textwrap.fill(response, 60))

In [22]:
prettyCypherChain("Đào tạo chính quy là gì?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (n:Khái_niệm {id: "Đào Tạo Chính Quy"}) RETURN n.định_nghĩa

Full Context:
[{'n.định_nghĩa': None}]

> Finished chain.
Tôi không biết câu trả lời cho câu hỏi "Đào tạo chính quy là
gì?".
